# This notebook is used to remove asphalt from the Natura 2000 region of Coepelduynen.
Weirdly enough parking spaces and some roads are still included in the official polygon region of Coepelduynen.
We made the decision to remove these ashpalt regions from the satellite images.

We combine here model results from a previous model which predicted asphalt supplement with handdrawn annotations.

In [ ]:
%matplotlib notebook
import glob
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Note this .shp file is the result of a model which predicted Asphalt but was not fully correct
gdf = gpd.read_file("E:/output/Coepelduynen_segmentations/20230908_110020_PNEO-04_1_1_30cm_RD_12bit_RGBNED_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_re_ndvi.shp")
gdf = gdf[gdf['label'] == "Asphalt"]
gdf = gdf.explode()
gdf['area'] = gdf['geometry'].area
gdf['perimeter'] = gdf['geometry'].length

In [ ]:
# Filter out the faulty ashpalt regions.
gdf[(gdf["area"] > 50) & (gdf['perimeter'] > 150) ].unary_union

In [ ]:
# Create a new GeoDataFrame with the resulting MultiPolygon
gdf_save = gpd.GeoDataFrame({'geometry': [gdf[(gdf["area"] > 50) & (gdf['perimeter'] > 150) ].unary_union]}, geometry='geometry', crs=gdf.crs)

In [ ]:
gdf_save.to_file("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/asfalt_detection_coepelduynen.shp")

### Now merge with handdrawn annotated data

In [ ]:
annotated_gdf = gpd.read_file("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/asfalt_annotated_coepelduynen.geojson")
annotated_gdf['geometry'] = annotated_gdf['geometry'].buffer(0)

In [ ]:
# Check the result
gdf_save.append(annotated_gdf,ignore_index=True).drop("id",axis=1).unary_union

In [ ]:
gdf_save = gpd.GeoDataFrame({'geometry': [gdf_save.append(annotated_gdf,ignore_index=True).drop("id",axis=1).unary_union]}, geometry='geometry', crs=gdf.crs)
gdf_save.to_file("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/asfalt_coepelduynen.shp")

### Extract from the original file

In [ ]:
def __make_the_crop(load_shape, raster_path, raster_path_cropped, plot):
    """
        This crops the sattelite image with a chosen shape.

        TODO: Make this accept a object of geopandas or shapely and crs independant.
        @param load_schape: path to a geojson shape file.
        @param raster_path_wgs: path to the raster .tiff file.
        @param raster_path_cropped: path were the cropped raster will be stored.
    """
    geo_file = gpd.read_file(load_shape)
    src = rasterio.open(raster_path)
    print('raster path opened')
    # Change the crs to rijks driehoek, because all the satelliet images are in rijks driehoek
    if geo_file.crs != 'epsg:28992':
        geo_file = geo_file.to_crs(epsg=28992)

    out_image, out_transform = rasterio.mask.mask(src,geo_file['geometry'], crop=True, filled=True)
    out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    print('convert to RD')

    src.close()
    with rasterio.open(raster_path_cropped, "w", **out_meta) as dest:
            dest.write(out_image)
            dest.close()

    if plot:
        print("Plotting data for:"+raster_path_cropped+"-----------------------------------------------------")
        # TODO: Make this optional to plot.
        src = rasterio.open(raster_path_cropped)
        plot_out_image = np.clip(src.read()[2::-1],
                        0,2200)/2200 # out_image[2::-1] selects the first three items, reversed

        plt.figure(figsize=(10,10))
        rasterio.plot.show(plot_out_image,
            transform=src.transform)
        logging.info(f'Plotted cropped image {raster_path_cropped}')
        src.close()

In [ ]:
coepelduynen_gdf = gpd.read_file("C:/Users/pzhadmin/Documents/natura2000_coepelduynen.geojson")
coepelduynen_gdf =coepelduynen_gdf.to_crs("EPSG:28992")
diff_gdf = coepelduynen_gdf.difference(gdf_save)
diff_gdf.to_file("C:/Users/pzhadmin/Documents/natura2000_coepelduynen_no_asphalt.geojson")

In [ ]:
__make_the_crop("C:/Users/pzhadmin/Documents/natura2000_coepelduynen_no_asphalt.geojson", 'E:/data/coepelduynen/20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBNED_NoordwijkAanZee_natura2000_coepelduynen_cropped_ndvi_re_ndvi.tif','E:/data/coepelduynen/20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBNED_NoordwijkAanZee_natura2000_coepelduynen_cropped_ndvi_re_ndvi_no_asphalt_crop.tif', True)

In [ ]:
for file in glob.glob("E:/data/coepelduynen/*ndvi*.tif"):
    file = file.replace("\\","/")
    print(file)
    print("---")
   
    print( file.replace(".tif","_asphalt_crop.tif"))
    
    __make_the_crop("C:/Users/pzhadmin/Documents/natura2000_coepelduynen_no_asphalt.geojson",file,file.replace(".tif","_asphalt_crop.tif"), False)